In [4]:
import numpy as np
from pandas import DataFrame, ExcelWriter, read_excel
from collections import OrderedDict

def dam_storage_sdr_annual(dtl_a, dsc_a, dda_a, dste_a, dnuc_a):
     
    """
    This function calculates the annual variations of 
    (a) dams' sediment input, Sediment Trapping Efficiency (STE), and storage capacity;
    (b) catchment's Sediment Delivery Ratio
    """
    
    ssi_tv = np.sum(ssi_a)  # stream sediment input total value (ton)
    ssi_ca = np.copy(ssi_a)  # copied array of stream sediment input
    
    # store the connected order of dams' location
    dlco_a = np.empty(len(dtl_a), dtype=int)
    for i_v1, v1 in enumerate(dtl_a):
        dlco_a[i_v1] = np.array([i_v2 for i_v2, v2 in enumerate(slco_a) if v1 in v2])
    
    # argument sort of dams' stream ID connected order and sorting other data 
    dlco_as = np.argsort(dlco_a) 
    dtl_sa = dtl_a[dlco_as] 
    dsc_sa = dsc_a[dlco_as] 
    dste_sa = dste_a[dlco_as] 
    dnuc_sa = dnuc_a[dlco_as]
    
    dsd_v = 0  # initial value of cumulative sediment deposition in dams
    
    # Store dams' sediment input, storage capacity and STE, and catchmnet's SDR value
    dsi_sa = np.empty(len(dtl_a))
    for i_v1, v1 in enumerate(dtl_sa): 
        v1_suc = sluc_a[v1 - 1]  
        if len(v1_suc) == 0:  # if dam's upstream connection is not found
            v1_dsi = ssi_ca[v1-1]
            dsi_sa[i_v1] = v1_dsi  
            v1_dsd = v1_dsi * dste_sa[i_v1]  
            dsd_v += v1_dsd 
            dsc_sa[i_v1] -= (v1_dsd*csw_kg) / s_sg 
            ssi_ca[v1-1] -= v1_dsd
        else:  # if dam's upstream connection is found
            v1_udc = np.array([v2 for v2 in dtl_sa if v2 in v1_suc and v2 != v1])  
            if len(v1_udc) == 0:  # if upstream dam's upstream segments are not found
                v1_dsi = np.sum(ssi_ca[sluc_a[v1-1] - 1])
                dsi_sa[i_v1] = v1_dsi  
                v1_dsd = v1_dsi * dste_sa[i_v1]  
                dsd_v += v1_dsd  
                dsc_sa[i_v1] -= (v1_dsd*csw_kg) / s_sg 
                ssi_ca[v1-1] = v1_dsi - v1_dsd  
            else:  # if upstream dam's upstream segments are found
                v1_rus = np.concatenate([sluc_a[v3-1] if len(sluc_a[v3-1]) != 0 else np.array([v3]) 
                                         for v3 in v1_udc])  # remove upstream segments
                v1_urus = np.unique(v1_rus) 
                v1_auc = np.setdiff1d(v1_suc, v1_urus)  
                v1_dsi = np.sum(ssi_ca[v1_auc-1]) + np.sum(ssi_ca[dtl_a[dnuc_sa[i_v1]] - 1]) 
                dsi_sa[i_v1] = v1_dsi  
                v1_dsd = v1_dsi * dste_sa[i_v1]
                dsd_v += v1_dsd  
                dsc_sa[i_v1] -= (v1_dsd*csw_kg) / s_sg 
                ssi_ca[v1-1] = v1_dsi - v1_dsd  
    
    # argument sort of dams' stream ID and corresponding sort of other data
    dtl_2as = np.argsort(dtl_sa)
    dsi_a = dsi_sa[dtl_2as] 
    dsc_a = dsc_sa[dtl_2as]  
    dste_a = dam_ste(dsc_a, dda_a) 
    
    # Sediment Delivery Ratio
    sdr_v = (ssi_tv-dsd_v) / ssi_tv
    
    return dsi_a, dsc_a, dste_a, sdr_v  # return the outputs


def dam_next_upstream_connection(dtl_a):
    
    """
    This function calculates the next upstream connection of dams
    """
    
    # store the dams' all upstream connection
    duc_a = np.empty(len(dtl_a), dtype=object) 
    for i_v1, v1 in enumerate(dtl_a): 
        v1_sluc = sluc_a[v1-1]
        if len(v1_sluc) != 0: 
            duc_a[i_v1] = np.array([np.asscalar(np.where(dtl_a == v2)[0]) 
                                    for v2 in dtl_a 
                                    if v2 in v1_sluc and v2 != v1])
        else: 
            duc_a[i_v1] = v1_sluc
    
    # store the dams' next upstream connection only
    dnuc_a = np.empty(len(dtl_a), dtype=object) 
    for i_v1, v1 in enumerate(duc_a):  
        if len(v1) == 0:  
            dnuc_a[i_v1] = v1 
        else:  
            v1_uuc = [duc_a[v2] for v2 in v1 if len(duc_a[v2]) != 0]
            if len(v1_uuc) != 0:  
                v1_ucd = np.unique(np.concatenate(v1_uuc)) 
                dnuc_a[i_v1] = np.setdiff1d(v1, v1_ucd) 
            else:      
                dnuc_a[i_v1] = v1 
    
    return dnuc_a  # return the output


def dam_ste(dsc_a, dda_a):
    
    """
    This function calculates the dams' Sediment Trapping Efficiency (STE) 
    """
    
    dste_a = 1 - (1 / (1 + ((0.0021*0.1*dsc_a) / dda_a))) 
    
    return dste_a  # return the output


def dam_output_data(dtl_a, dda_a, dsc_a):
    
    """
    This function calculates:
    (a) the arrays of dams' annual storage capacity variation
    (b) life expectancy of each dam, 
    (c) array of Sediment Delivery Ratio (SDR) up to the maximum life expectancy of all dams 
    """
    
    did_a = np.array(np.arange(len(dtl_a)), dtype=int)  # array of dams' ID
    dste_a = dam_ste(dsc_a, dda_a)  # array of dams's initial STE
    dnuc_a = dam_next_upstream_connection(dtl_a)  # array of dams' initial next upstream connection
    dfo_a = np.empty(0, dtype=int)  # an empty array to store the order of inactive dam(s)
    
    dscv_l = []  # an empty list to store dams' storage capacity variation
    sdr_a = np.empty(0)  # an empty array to store annual SDR variation
    dsv_m = np.array([dsc_a])  # dams' storage capacity variation matrix with first row as initial value  
    
    # store annual variaton of dams' storage capacity and catchment's SDR
    yr_i = 0  # initial year
    while True:
        yr_i += 1 
        dsi_a, dsc_a, dste_a, sdr_v = dam_storage_sdr_annual(dtl_a, dsc_a, dda_a, dste_a, dnuc_a)
        dsv_m = np.vstack((dsv_m, dsc_a))
        sdr_a = np.append(sdr_a, sdr_v) 
        if yr_i >= dle_max:
            fdi_a = np.where(dtl_a > 0)[0]
        elif np.any((dsc_a <= 0) | (dste_a < ste_tv)):
            fdi_a = np.where((dsc_a <= 0) | (dste_a < ste_tv))[0]
        else:
            continue
        dfo_a = np.append(dfo_a, did_a[fdi_a])
        [dscv_l.append(dsv_m[:, v1]) for v1 in fdi_a]  
        dsv_m = np.delete(dsv_m, fdi_a, axis=1) 
        did_a = np.delete(did_a, fdi_a)
        dtl_a = np.delete(dtl_a, fdi_a) 
        dsc_a = np.delete(dsc_a, fdi_a) 
        dste_a = np.delete(dste_a, fdi_a)
        dnuc_a = dam_next_upstream_connection(dtl_a)  
        dda_a = dam_drainage_area(dtl_a)
        if len(did_a) == 0:
            break
        else:
            continue
    
    # sorted dams' annual storage variation and life expectancy
    dsv_a = (dscv_l[v1] for v1 in np.argsort(dfo_a)) 
    dpvsv_a = (np.delete(v1, np.where(v1 < 0)[0]) for v1 in dsv_a)  
    dnsv_a = np.array([v1/v1[0] for v1 in dpvsv_a])  
    dle_a = np.array([(len(v1)-1) for v1 in dnsv_a])  

    return dnsv_a, dle_a, sdr_a  # return the outputs


def dam_drainage_area(dtl_a):
    
    """
    This function calculates the dams' initial drainage area 
    based on their locations in the stream path
    """
    
    # store the connected order of dams' location
    dlco_a = np.empty(len(dtl_a), dtype=int)  
    for i_v1, v1 in enumerate(dtl_a):
        dlco_a[i_v1] = np.array([i_v2 for i_v2, v2 in enumerate(slco_a) if v1 in v2])
    
    # argument sort of dams' connected order and sorting other data
    dlco_as = np.argsort(dlco_a)
    dlco_sa = dtl_a[dlco_as]  
    
    # store dams' flow accumulation by dams' stream ID connected order
    dtlfa_sa = np.empty(len(dtl_a))
    for i_v1, v1 in enumerate(dlco_sa): 
        v1_sluc = sluc_a[v1-1]  
        v1_udc = [v2 for v2 in dlco_sa if v2 in v1_sluc and v2 != v1] 
        if len(v1_udc) == 0:   
            dtlfa_sa[i_v1] = fa_a[v1-1] + 1 
        else:  
            v1_udci = [np.asscalar(np.where(dlco_sa == v3)[0]) for v3 in v1_udc]  
            v1_udfa = np.sum([dtlfa_sa[v4] for v4 in v1_udci])   
            dtlfa_sa[i_v1] = (fa_a[v1-1]-v1_udfa) + 1  
    
    # dams drainage area (km^2) according to dams' targeted location 
    dtlfa_a = dtlfa_sa[np.argsort(dlco_sa)]
    dda_a = dtlfa_a * pa_v 
    
    return dda_a  # return the output


def stream_connection_structure():
    
    """
    This function calculates: 
    (a) all the connnected upstream segments for each stream segment,
    (b) connected order of stream segments from downstream to upstream
    """
    
    # an empty array to store stream link upstream connection
    sluc_a = np.empty(len(sid_a), dtype=object)
    
    # an empty list to store connected order of stream segments from downstream to upstream
    slco_l = []  
    
    # end segment
    sl_mds = sid_a[np.where(nsid_a == (len(nsid_a) + 1))[0]]
    sl_mdsi = np.asscalar(np.where(sid_a == sl_mds)[0])
    sluc_a[sl_mdsi] = sl_mds
    slco_l.append(sl_mds)
    
    while True:  
        if len(sl_mds) == 1:  # detemine next downstream segments of end segment and store the values
            sl_nuci = np.where(nsid_a == sl_mds)[0]
            sl_mds = sid_a[sl_nuci]
            sluc_a[sl_mdsi] = np.append(sluc_a[sl_mdsi], sl_mds)
            slco_l.append(sl_mds) 
        else:  # update next donwstream segments and store the values
            sl_nuci = np.array([np.where(nsid_a == v1)[0] for v1 in sl_mds])
            sl_nuc = [sid_a[v2] if len(v2) != 0 else v2 for v2 in sl_nuci] 
            for v3, v4 in zip(sl_mds, sl_nuc):
                if len(v4) != 0:
                    sluc_a[v3-1] = np.append(sid_a[v3-1], v4)
                else:
                    sluc_a[v3-1] = v4
            for v5 in sl_mds:  # iterate updated next downstream segments and store the values
                sl_uci = np.array([i_v6 for i_v6, v6 in enumerate(sluc_a) 
                                   if v6 is not None and v5 in v6 and i_v6 != v5 - 1])
                for v7 in sl_uci:
                    v7_uuc = np.append(sluc_a[v7], sluc_a[v5-1])
                    v7_usp = np.unique(v7_uuc, return_index=True)
                    sluc_a[v7] = v7_uuc[np.sort(v7_usp[1])] 
            sl_nenuc = [sid_a[v8] for v8 in sl_nuci if len(v8) != 0]
            if len(sl_nenuc) != 0:  # continue if non-empty next upstream connection is found
                sl_mds = np.concatenate(sl_nenuc)
                slco_l.append(sl_mds)
            else:  # otherwise break the loop 
                break 
    
    # connected order of stream segments from downstream to upstream
    slco_a = np.array(slco_l)[::-1]  
    
    return sluc_a, slco_a  # return the outputs


def stodym_output_write(excel_file=False, stodym_fp="StoDyM_Output.xlsx"):
    
    """
    This function gives the output of StoDyM and write the output data in excel file if option is given
    """
    
    didcn_l = ["D"+str(v3) for v3 in range(len(dtl_a))]  # Dams' ID coulmn list
    
    # Dams' locations, storage capacity, drainage area and life expectancy  
    dccn_l = ["DID", "SIN", "ISC(m^3)", "IDA(%)", "LE(yr)"]
    dc_df = DataFrame.from_records(zip(didcn_l, dtl_a, dsc_a, ddap_a, dle_a), columns=dccn_l)
    
    # Dams' normalized storage capacity variation
    nsv_od = OrderedDict([(v1, v2) for v1, v2 in zip(didcn_l, dnsv_a)]) 
    nsv_df = DataFrame.from_dict(nsv_od, orient="index").T
    nsv_df.insert(0, "Year", range(1+np.amax(dle_a)))

    # SDR variation
    sdr_od = OrderedDict([(v1, v2) for v1, v2 in zip(["Year", "SDR"], [1+np.arange(len(sdr_a)), sdr_a])])
    sdr_df = DataFrame.from_dict(sdr_od)
    
    # Dam deployment summary with objective values
    dsscn_l = ["ITSC(m^3)", "ITDA(%)", "LE_A(yr)"] + ocn_l 
    if len(np.where(dle_a >= dle_min)[0]) >= 2:
        sd_m = np.array([v1[:dle_min] for v1, v2 in zip(dnsv_a, dle_a) 
                            if v2 >= dle_min]) 
        sdm_a = np.mean(sd_m, axis=0)
        sdd_max = np.amax([np.mean(np.power(v1-sdm_a, 2))**0.5 for v1 in sd_m])
        dds_l = [np.sum(dsc_a), np.sum(ddap_a), np.mean(dle_a), 
                 np.mean(dle_a)/dle_max, sdd_max, sdr_a[0], sdr_a[dle_min-1]]
    else:
        sdd_max = "NA" 
        dds_l = [np.sum(dsc_a), np.sum(ddap_a), np.mean(dle_a), 
                 np.mean(dle_a)/dle_max, sdd_max, sdr_a[0], sdr_a[-1]]
    dds_df = DataFrame.from_records([dds_l], columns=dsscn_l).T
    
    # StoDyM output ordered dictionary
    stodym_od = OrderedDict([(v1, v2) for v1, v2 in zip(stodym_sn, [dc_df, nsv_df, sdr_df, dds_df])])
    
    # if the data need to be saved in excel file
    if excel_file == True:       
        stodymxl_wo = ExcelWriter(stodym_fp, engine="xlsxwriter") 
        dc_df.to_excel(stodymxl_wo, sheet_name=stodym_sn[0], index=False)
        nsv_df.to_excel(stodymxl_wo, sheet_name=stodym_sn[1], index=False)
        sdr_df.to_excel(stodymxl_wo, sheet_name=stodym_sn[2], index=False) 
        dds_df.to_excel(stodymxl_wo, sheet_name=stodym_sn[3], index=True, header=False)
        stodymxl_wo.save()

    return stodym_od

c_pn, pa_v = (12419, 20**2/10**6)  # number of pixel in catchment area and pixel area in km^2
ste_tv = 0.1  # dam Sediment Trapping Efficiency (STE) threshold value
ca_km2 = c_pn * pa_v  # catchment area (km^2)
dle_min, dle_max = (20, 30)  # dam life expectancy minimum and maxiumum values
csw_kg, s_sg = (907.185, 2650)  # connversion of sediment unit to Kg and sediment specific graity 

sc_fn = "Shejiagou_Stream_Characteristics.xlsx"  # stream characteristics file name
sc_df = read_excel(sc_fn, sheet_name="SC_Data")  # stream characteristics dataframe
sid_a = np.array(sc_df["SID"], dtype=int)  # stream ID array
nsid_a = np.array(sc_df["NSID"], dtype=int)  # next stream ID array
fa_a = np.array(sc_df["DFA"], dtype=int)  # flow accumulation array
ssi_a = np.array(sc_df["SSI"], dtype=float)  # stream sediment input array

sluc_a, slco_a = stream_connection_structure()  # stream link upstream connection and connected order

dtl_a = np.array([9, 11, 22, 27, 34])  # dam targeted location stream ID
dsc_a = 10**3*np.array([43, 52, 59, 58, 28], dtype=float)  # dams' initial storage capacity array
dda_a = dam_drainage_area(dtl_a)  # dam drainage area array
ddap_a = 100 * (dda_a/ca_km2)  # dam initial drainage area percentage array
dnsv_a, dle_a, sdr_a = dam_output_data(dtl_a, dda_a, dsc_a)  # dam output data

ocn_l = ["O-LE", "O-SD", "O-SDR,ST", "O-SDR,LT"]  # objective column name list
stodym_sn = ["Dam_Characteristics", "Storage_Variation", "SDR_Variation", "Summary"]  # sheet name
stodym_od = stodym_output_write(excel_file=False, stodym_fp="StoDyM_Output.xlsx")  # StoDyM ordered dictionary
stodym_od


OrderedDict([('Dam_Characteristics',   DID  SIN  ISC(m^3)     IDA(%)  LE(yr)
              0  D0    9   43000.0  15.492391      28
              1  D1   11   52000.0  11.570980      25
              2  D2   22   59000.0  29.116676      24
              3  D3   27   58000.0  15.274982      22
              4  D4   34   28000.0  17.336339      22),
             ('Storage_Variation',
                  Year        D0        D1        D2        D3        D4
              0      0  1.000000  1.000000  1.000000  1.000000  1.000000
              1      1  0.955023  0.952350  0.952711  0.956837  0.959683
              2      2  0.910211  0.904818  0.905466  0.913532  0.919346
              3      3  0.865580  0.857417  0.858269  0.870070  0.878984
              4      4  0.821145  0.810160  0.811131  0.826436  0.838591
              5      5  0.776924  0.763061  0.764058  0.782612  0.798158
              6      6  0.732939  0.716140  0.717064  0.738577  0.757673
              7      7  0.689213